In [ ]:
!pip install transformers datasets evaluate bert_score

In [ ]:
import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset
from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)
import torch
import random
import os

In [ ]:
# Ensuring reproducibility of the code.
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(42)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/NLP_Project_Folder/NLP_Dataset/train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/NLP_Project_Folder/NLP_Dataset/validation.csv')

print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("\n=== Train Data Sample ===")
display(train_df.head(5))
print("\n=== Validation Data Sample ===")
display(val_df.head(5))
print("\nTrain columns:", train_df.columns.tolist())
print("Validation columns:", val_df.columns.tolist())

Train shape: (9532, 31)
Validation shape: (1470, 32)

=== Train Data Sample ===


,hatespeech,csType,counterspeech,Suggest,Relevance,Aggressive,Complexity,Comments,source,claim,...,hatespeechTarget,powerDynamics,prompt_offensiveness,prompt_target_group,prompt_speaker_intent,prompt_power_dynamics,prompt_implication,prompt_emotional_reaction,prompt_cognitive_reaction,prompt_cs_generation
0,Maybe the UN could talk to those asian and afr...,Informative,The us is the second most polluting country in...,3,4.0,2.0,3.0,NaN,Human,The UN should focus on Asian and African natio...,...,asian_people,racial,Analyze the offensiveness of the statement: Ma...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
1,Maybe the UN could talk to those asian and afr...,Questioning,Doesn't everyone on the planet have a responsi...,,3.0,2.0,2.0,NaN,Human,The UN should focus on Asian and African natio...,...,asian_people,racial,Analyze the offensiveness of the statement: Ma...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
2,Maybe the UN could talk to those asian and afr...,Denouncing,The world would be a better place if people we...,1,1.0,2.0,2.0,NaN,Human,The UN should focus on Asian and African natio...,...,asian_people,racial,Analyze the offensiveness of the statement: Ma...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
3,Maybe the UN could talk to those asian and afr...,Positive,"You're right, ocean pollution is one of the ma...",3,4.0,2.0,2.0,NaN,Human,The UN should focus on Asian and African natio...,...,asian_people,racial,Analyze the offensiveness of the statement: Ma...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
4,Juice Jews are worse that nukes. Caption to a ...,Informative,Anti-semitism is a serious problem that we nee...,2,3.0,2.0,2.0,NaN,Human,Jews are worse than nukes.,...,jews,religion,Analyze the offensiveness of the statement: Ju...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...



=== Validation Data Sample ===


,index,hatespeech,csType,counterspeech,Suggest,Relevance,Aggressive,Complexity,Comments,source,...,hatespeechTarget,powerDynamics,prompt_offensiveness,prompt_target_group,prompt_speaker_intent,prompt_power_dynamics,prompt_implication,prompt_emotional_reaction,prompt_cognitive_reaction,prompt_cs_generation
0,21,"Women are basically childlike, they remain thi...",Questioning,How could you say something like this inspite ...,3.0,4.0,3.0,3.0,NaN,Human,...,women,gender,Analyze the offensiveness of the statement: Wo...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
1,50,Minorities vote to strip wealth and power from...,Denouncing,Aside from your statement having no factual ba...,3.0,4.0,3.0,2.0,NaN,Human,...,jews,religion,Analyze the offensiveness of the statement: Mi...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
2,141,We began to break down when we allowed women t...,Informative,Why is being 'soft' a problem? everyone is ent...,3.0,3.0,2.0,2.0,NaN,Human,...,women,gender,Analyze the offensiveness of the statement: We...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
3,163,Immigrants just dont want to integrate. Lets k...,Denouncing,They do. the reason why they find integration ...,3.0,4.0,4.0,2.0,NaN,Human,...,immigrants,immigration_and_citizenship,Analyze the offensiveness of the statement: Im...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
4,225,Muslims are asking too much! Their sharia has ...,Informative,This hoax about muslims wanting to ban dogs in...,3.0,5.0,3.0,2.0,NaN,Human,...,muslims,religion,Analyze the offensiveness of the statement: Mu...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...



Train columns: ['hatespeech', 'csType', 'counterspeech', 'Suggest', 'Relevance', 'Aggressive', 'Complexity', 'Comments', 'source', 'claim', 'centralTopic', 'speakerIntent', 'targetGroup', 'relevantPowerDynamics', 'hatespeechImplication', 'targetGroupEmotionalReaction', 'targetGroupCognitiveReaction', 'hatespeechOffensiveness', 'id', 'is_high_quality', 'hs_id', 'hatespeechTarget', 'powerDynamics', 'prompt_offensiveness', 'prompt_target_group', 'prompt_speaker_intent', 'prompt_power_dynamics', 'prompt_implication', 'prompt_emotional_reaction', 'prompt_cognitive_reaction', 'prompt_cs_generation']
Validation columns: ['index', 'hatespeech', 'csType', 'counterspeech', 'Suggest', 'Relevance', 'Aggressive', 'Complexity', 'Comments', 'source', 'claim', 'centralTopic', 'speakerIntent', 'targetGroup', 'relevantPowerDynamics', 'hatespeechImplication', 'targetGroupEmotionalReaction', 'targetGroupCognitiveReaction', 'hatespeechOffensiveness', 'id', 'is_high_quality', 'hs_id', 'hatespeechTarget', '

In [ ]:
# Loading BART Tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
#This function will be combining the hate speech and intent text, and then tokenize it using BART.
def preprocess_function(examples):
    combined_input = [
        f"intent: {it} hatespeech: {hs}"
        for it, hs in zip(examples['csType'], examples['hatespeech'])
    ]
    targets = examples["counterspeech"]
    # Tokenize input
    model_inputs = tokenizer(
        combined_input,
        max_length=256,
        truncation=True,
        padding="max_length"
    )
    # Tokenize targets (counterspeech)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=256,
            truncation=True,
            padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [ ]:
# Convert pandas to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/9532 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

In [ ]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
# This collator will pad dynamically at batch-time
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
bleu_metric = evaluate.load("bleu")
bertscore_metric = evaluate.load("bertscore")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    references = [[ref] for ref in decoded_labels]
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=references)
    bertscore_result = bertscore_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        lang="en"
    )
    return {
        "bleu": bleu_result["bleu"],
        "bertscore_f1": np.mean(bertscore_result["f1"])
    }

In [ ]:
from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,  # <--- now valid
    logging_steps=100,
    push_to_hub=False
)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-d069e7cf15ba>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ansh22083 (ansh22083-indraprastha-institute-of-information-technolo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3576, training_loss=0.41084552931305546, metrics={'train_runtime': 2572.4485, 'train_samples_per_second': 11.116, 'train_steps_per_second': 1.39, 'total_flos': 4359005939957760.0, 'train_loss': 0.41084552931305546, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print("\n=== Validation Evaluation Results ===")
print(eval_results)


=== Validation Evaluation Results ===
{'eval_loss': 0.2919559180736542, 'eval_bleu': 0.022827409129051245, 'eval_bertscore_f1': 0.8662013249737875, 'eval_runtime': 129.0097, 'eval_samples_per_second': 11.394, 'eval_steps_per_second': 1.426, 'epoch': 3.0}
